In [11]:
import pprint
import requests
import sys
import pandas as pd
from datetime import datetime

api_key = "9b656c6f5237727cdf1e35288a7afa8c"
geo_URL = 'http://api.openweathermap.org/geo/1.0/direct'

loc = [
  'Buenos Aires, Argentina',
  'Guangzhou, China',
  'Wichita, Kansas',
  'Niskayuna, New York',
  'Gwangmyeong, South Korea',
  'Taipei, Taiwan',
  'Nanaimo, British Columbia',
  'Chennai, India',
  'Barrington, Illinois',
  'Littleton, Colorado',
  'Peterhead, Scotland',
  'Vizag, India',
  'Des Moines, Iowa',
  'Beijing, China',
  'Killeen, Texas',
  'Morehead City, North Carolina'
]

df_wide = pd.DataFrame(index=loc)
######

for city in loc:
    geo = f'{geo_URL}?q={city}&limit=5&appid={api_key}'
    resp = requests.get( geo )

    if resp.status_code != 200: # Failure?
        print( f'Error geocoding {city}: {resp.status_code}' )
        sys.exit( 1 )

    # OpenWeatherMap returns a list of matching cities, up to the limit specified
    # in the API call; even if you only ask for one city (limit=5), it's still
    # returned as a 1-element list

    if len( resp.json() ) == 0: # No such city?
        print( f'Error locating city {city}; {resp.status_code}' )
        sys.exit( 2 )

    json = resp.json()
    if type( json ) == list: # List of cities?
        lat = json[ 0 ][ 'lat' ]
        lon = json[ 0 ][ 'lon' ]
    else: # Unknown city?
        print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
        sys.exit( 3 )

    # Use Peterhead's latitude and longitude to get its 5-day forecast in 3-hour blocks

    forecast_URL = 'http://api.openweathermap.org/data/2.5/forecast'
    forecast = f'{forecast_URL}?lat={lat}&lon={lon}&appid={api_key}'
    resp = requests.get( forecast )

    if resp.status_code != 200: # Failure?
        print( f'Error retrieving data: {resp.status_code}' )
        sys.exit( 4 )

    data = resp.json()
##### 

    data_list = []
    for i in range(len(data['list'])):
        forecast_data = data[ 'list' ][ i ]
        dt = datetime.strptime(forecast_data['dt_txt'], '%Y-%m-%d %H:%M:%S')
        temp_max = forecast_data['main']['temp_max']
        temp_min = forecast_data['main']['temp_min']
        data_list.append({
            'city': city,
            'datetime': dt,
            'temp_max': temp_max,
            'temp_min': temp_min
        })

    df = pd.DataFrame(data_list)
#####

    df_grouped = df.groupby(pd.Grouper(key='datetime', axis=0, freq='D')).aggregate({
        'city': lambda x: x.iloc[0],
        'temp_max': 'max',
        'temp_min': 'min'})
   
    df_grouped['min_avg'] = df_grouped['temp_min'].mean()
    df_grouped['max_avg'] = df_grouped['temp_max'].mean()
#####

    for i, day in enumerate(df_grouped.index, start=0):
        if i>4:
            break
        df_wide.loc[city, f'Min_{i}'] = df_grouped.loc[day, 'temp_min']
        df_wide.loc[city, f'Max_{i}'] = df_grouped.loc[day, 'temp_max']


df_wide = df_wide.drop(columns=['Min_0', 'Max_0'])
df_wide = df_wide.iloc[:] - 273.15
df_wide = df_wide.reset_index()
df_wide = df_wide.rename({'index': 'City'}, axis='columns')
df_wide['Min_Avg'] = df_wide[['Min_1', 'Min_2', 'Min_3', 'Min_4']].mean(axis=1).round(2)
df_wide['Max_Avg'] = df_wide[['Max_1', 'Max_2', 'Max_3', 'Max_4']].mean(axis=1).round(2)
df_wide

,City,Min_1,Max_1,Min_2,Max_2,Min_3,Max_3,Min_4,Max_4,Min_Avg,Max_Avg
0,"Buenos Aires, Argentina",5.17,10.08,6.24,9.76,7.10,10.54,8.57,12.30,6.77,10.67
1,"Guangzhou, China",28.58,36.35,29.24,34.40,28.82,36.25,27.85,36.75,28.62,35.94
2,"Wichita, Kansas",25.49,37.65,24.91,39.08,27.75,40.78,27.90,40.39,26.51,39.48
3,"Niskayuna, New York",20.37,29.22,19.51,31.73,17.78,31.76,19.49,35.13,19.29,31.96
4,"Gwangmyeong, South Korea",23.11,30.27,23.82,30.84,24.90,32.40,22.38,28.00,23.55,30.38
5,"Taipei, Taiwan",27.90,36.79,27.07,35.80,26.79,34.98,27.10,35.89,27.22,35.87
6,"Nanaimo, British Columbia",14.77,22.46,15.22,22.42,15.48,22.81,15.81,20.95,15.32,22.16
7,"Chennai, India",29.00,34.97,28.58,33.17,28.28,30.47,27.67,32.60,28.38,32.80
8,"Barrington, Illinois",16.61,28.09,18.39,29.66,22.67,30.87,23.74,30.34,20.35,29.74
9,"Littleton, Colorado",19.99,38.45,21.21,38.83,21.87,40.15,23.05,37.57,21.53,38.75
